<a href="https://colab.research.google.com/github/ShogoNoguchi/LaBraM_fork/blob/main/%EF%BC%88%E9%80%B2%E6%8D%97%EF%BC%89%EF%BD%9E%E5%A4%A7%E8%A6%8F%E6%A8%A1%E6%A7%8B%E9%80%A0%E3%81%AE%E7%89%B9%E5%AE%9A%E3%81%A8%E4%BF%AE%E6%AD%A3%E9%A0%85%E7%9B%AE%E3%81%AE%E7%99%BA%E8%A6%8B%EF%BD%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

文責：群馬大学　野口翔伍

題：（進捗）～大規模構造の特定と修正項目の発見～

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


論文を熟読し、GitHubのREAD MEを読んで課題1つ目を処理する方法を調べた。
TUAB・TUEVどちらのデータセットを使っても良いと思ったが、どちらもTemple Universityへの要求方法が不明なため、質問済。
➡のちに、これらは使わず、SEED-Vを用いた感情分類をすることが判明。

リポジトリを[/content/drive/My Drive/LaBraM]へダウンロード済 12/2

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# 保存先ディレクトリの設定
import os
drive_path = '/content/drive/My Drive/LaBraM'  # Drive内に保存するフォルダ名
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# リポジトリをクローン
repo_url = "https://github.com/935963004/LaBraM.git"  # GitHubのURL
%cd /content
!git clone {repo_url} labram_repo
!cp -r labram_repo/* "{drive_path}"

# ディレクトリ構造を表示
import os

def display_directory_structure(path, level=0):
    indent = "  " * level
    print(f"{indent}{os.path.basename(path)}/")
    if os.path.isdir(path):
        for item in os.listdir(path):
            display_directory_structure(os.path.join(path, item), level + 1)

print("ディレクトリ構造:")
display_directory_structure(drive_path)

# 推奨ファイルを特定するために 'checkpoints' フォルダの内容を表示
checkpoints_dir = os.path.join(drive_path, 'checkpoints')
if os.path.exists(checkpoints_dir):
    print("\n[checkpoints]フォルダの内容:")
    for file in os.listdir(checkpoints_dir):
        print(f"  {file}")
else:
    print("\n[checkpoints]フォルダが存在しません。")


Mounted at /content/drive
/content
Cloning into 'labram_repo'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 124 (delta 49), reused 45 (delta 42), pack-reused 57 (from 1)
Receiving objects: 100% (124/124), 167.16 MiB | 26.39 MiB/s, done.
Resolving deltas: 100% (55/55), done.
Updating files: 100% (27/27), done.
ディレクトリ構造:
LaBraM/
  checkpoints/
    labram-base.pth/
    vqnsp.pth/
  data_processor/
    data_preprocess.py/
    dataset.py/
  dataset_maker/
    make_TUAB.py/
    make_TUEV.py/
    make_h5dataset_for_pretrain.py/
    shock/
      __init__.py/
      utils/
        __init__.py/
        eegUtils.py/
        h5.py/
        ringBuffer.py/
  engine_for_finetuning.py/
  engine_for_pretraining.py/
  engine_for_vqnsp.py/
  labram.png/
  modeling_finetune.py/
  modeling_pretrain.py/
  modeling_vqnsp.py/
  norm_ema_quantizer.py/
  optim_factory.py/
  README.md/
  requirements.txt/
  r

大規模コードの木構造を特定 12/3

ファインチューニング時にはGiTHubの以下のコマンドにより
run_class_finetuning.pyが最初に起動される。

In [1]:
OMP_NUM_THREADS=1 torchrun --nnodes=1 --nproc_per_node=8 run_class_finetuning.py \
        --output_dir ./checkpoints/finetune_tuab_base/ \
        --log_dir ./log/finetune_tuab_base \
        --model labram_base_patch200_200 \
        --finetune ./checkpoints/labram-base.pth \    #baseモデルの重み
        --weight_decay 0.05 \
        --batch_size 64 \
        --lr 5e-4 \
        --update_freq 1 \
        --warmup_epochs 3 \
        --epochs 30 \
        --layer_decay 0.65 \
        --drop_path 0.1 \
        --dist_eval \
        --save_ckpt_freq 5 \
        --disable_rel_pos_bias \
        --abs_pos_emb \
        --dataset TUAB \   #データセット名の指定
        --disable_qkv_bias \
        --seed 0

SyntaxError: invalid syntax (<ipython-input-1-ce411afbb617>, line 1)

run_class_finetuning.pyから、以下の依存関係を特定した。

In [1]:
run_class_finetuning.py
├── argparse (標準ライブラリ)
├── datetime (標準ライブラリ)
├── numpy (外部ライブラリ)
├── time (標準ライブラリ)
├── torch (外部ライブラリ)
│   ├── torch.backends.cudnn
│   ├── torch.utils.data
│   └── torch.nn
├── json (標準ライブラリ)
├── os (標準ライブラリ)
├── pathlib.Path (標準ライブラリ)
├── collections.OrderedDict (標準ライブラリ)
├── timm (外部ライブラリ)
│   ├── timm.data.mixup.Mixup
│   ├── timm.models.create_model   #モデル構造を入れて、モデルを出力
│   ├── timm.loss.LabelSmoothingCrossEntropy
│   ├── timm.loss.SoftTargetCrossEntropy
│   └── timm.utils.ModelEma
├── optim_factory.py     #最適化関連
│   ├── create_optimizer
│   ├── get_parameter_groups
│   └── LayerDecayValueAssigner
├── engine_for_finetuning.py  #ファインチューニング用のエンジン
│   ├── train_one_epoch
│   └── evaluate
├── utils.py
│   ├── NativeScalerWithGradNormCount   #学習率スケーリング
│   ├── init_distributed_mode
│   ├── create_ds_config
│   ├── get_rank
│   ├── cosine_scheduler
│   ├── auto_load_model
│   ├── save_model
│   ├── is_main_process
│   ├── prepare_TUAB_dataset   #ここを変える必要がある。 utils内のprepare_oo_dataset系
│   ├── prepare_TUEV_dataset
│   ├── TensorboardLogger
│   └── load_state_dict
├── modeling_finetune.py    #ここを調べるとファインチューニングのモデル構造が分かる・変更できる。
│   └── モデルの定義
├── scipy.interpolate (外部ライブラリ)  #これはなんでインポートされてるか分からない...


SyntaxError: invalid character '├' (U+251C) (<ipython-input-1-47fd71e04f21>, line 2)

run_class_finetuning.pyの実行時の関数呼び出し順序（木構造）

In [ ]:
run_class_finetuning.py
└── main(args, ds_init)
    ├── utils.init_distributed_mode(args)
    ├── デバイスとシードの設定
    ├── cudnn.benchmark = True
    ├── dataset_train, dataset_test, dataset_val, ch_names, metrics = get_dataset(args)
    │   └── prepare_TUAB_dataset または prepare_TUEV_dataset（utils.py内）
    ├── サンプラーとデータローダーの設定
    │   ├── torch.utils.data.DistributedSampler（トレーニング用）
    │   ├── torch.utils.data.SequentialSampler（検証・テスト用）
    │   └── torch.utils.data.DataLoader（データローダー作成）
    ├── ロガーの設定（TensorboardLogger）
    ├── model = get_models(args)
    │   └── timm.models.create_model による。create_modelの中で args.modelが渡されてmodeling_finetune.py の中のモデル（labram_base_patch200_200）構造を取得する。
    ├── モデルの初期化と読み込み   #モデルの重みがここで読まれる。
    │   ├── 事前学習済みモデルのロード（args.finetune が指定されている場合）
    │   ├── モデルの重みの調整（必要に応じて）
    │   └── モデルをデバイスに転送（model.to(device)）
    ├── モデルEMAの設定（必要に応じて）
    ├── DistributedDataParallel の設定（分散学習の場合）
    ├── オプティマイザとスケジューラの設定
    │   ├── optimizer = create_optimizer(...)
    │   ├── loss_scaler = NativeScaler()
    │   ├── lr_schedule_values = utils.cosine_scheduler(...)
    │   └── wd_schedule_values = utils.cosine_scheduler(...)
    ├── 損失関数の設定（criterion）
    ├── utils.auto_load_model(...)（チェックポイントの自動ロード）
    ├── トレーニングループ開始（エポックごとに繰り返し）
    │   ├── data_loader_train.sampler.set_epoch(epoch)
    │   ├── train_stats = train_one_epoch(...)
    │   │   ├── モデルをトレーニングモードに設定（model.train()）
    │   │   ├── バッチごとのトレーニングループ
    │   │   │   ├── 入力データとラベルを取得
    │   │   │   ├── optimizer.zero_grad()
    │   │   │   ├── 出力の計算（outputs = model(inputs)）
    │   │   │   ├── 損失の計算（loss = criterion(outputs, targets)）
    │   │   │   ├── loss_scaler(loss, optimizer, ...)
    │   │   │   └── 学習率と重み減衰の更新
    │   │   └── 統計情報の収集
    │   ├── モデルの保存（必要に応じて）
    │   ├── 検証（data_loader_val がある場合）
    │   │   ├── val_stats = evaluate(...)
    │   │   │   ├── モデルを評価モードに設定（model.eval()）
    │   │   │   ├── バッチごとの評価ループ
    │   │   │   │   ├── 入力データとラベルを取得
    │   │   │   │   ├── 出力の計算（outputs = model(inputs)）
    │   │   │   │   ├── 損失の計算（loss = criterion(outputs, targets)）
    │   │   │   │   └── 統計情報の収集（metrics の計算）
    │   │   └── 検証結果の表示とログ記録
    │   ├── テスト（data_loader_test がある場合）
    │   │   ├── test_stats = evaluate(...)
    │   │   └── テスト結果の表示とログ記録
    │   ├── 最良モデルの保存（必要に応じて）
    │   └── エポックごとのログの保存
    └── トレーニング時間の計測と表示


分類対象の数を変えたいときはモデルのヘッドを変えないといけない、まずモデル構造の変えかたを調べよう。


ーーーモデル構造についてーーー

modeling_finetune.pyの中でtimm の @register_model デコレーターを用いて、カスタムモデル(labram_base_patch200_200)が登録されています。

args.model = 'labram_base_patch200_200'となってる影響で、

get_model関数＝create_modelの引数をargs.model とすると。labram_base_patch200_200 構造をtimmのcreate_modelが作ってくれる仕組み。
以下modeling_finetuneで独自モデルが登録されてる様子。

（NeuralTransformerはLaBramの本体的クラス）

In [ ]:
@register_model
def labram_base_patch200_200(pretrained=False, **kwargs):
    model = NeuralTransformer(
        patch_size=200, embed_dim=200, depth=12, num_heads=10, mlp_ratio=4,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model




ーーーーーーーーーーーーーー

run_class_finetuning.pyの
main関数内でモデルは以下のようにロードされる。
以下のコードは、モデルの構造（modeling_finetune.py で定義された構造）に合わせてチェックポイントからロードした重みを調整する仕組みになっている。
特に、ロードした重みの形状が現在のモデル構造と一致しない場合には、該当部分の重みを削除して適用されないようにしている点が特徴。

つまり、元のLaBramの自己教師モデルの重みのヘッド部分を強制的に消して、クラス分類へ適用するようになってる。


In [ ]:
model = get_models(args)  #モデル生成。 get_modelsはrun_class_finetuning中で定義

    patch_size = model.patch_size
    print("Patch size = %s" % str(patch_size))
    args.window_size = (1, args.input_size // patch_size)
    args.patch_size = patch_size

    if args.finetune:
        if args.finetune.startswith('https'):
            checkpoint = torch.hub.load_state_dict_from_url(
                args.finetune, map_location='cpu', check_hash=True)
        else:
            checkpoint = torch.load(args.finetune, map_location='cpu')

        print("Load ckpt from %s" % args.finetune)
        checkpoint_model = None
        for model_key in args.model_key.split('|'):
            if model_key in checkpoint:
                checkpoint_model = checkpoint[model_key]
                print("Load state_dict by model_key = %s" % model_key)
                break
        if checkpoint_model is None:
            checkpoint_model = checkpoint
        if (checkpoint_model is not None) and (args.model_filter_name != ''):
            all_keys = list(checkpoint_model.keys())
            new_dict = OrderedDict()
            for key in all_keys:
                if key.startswith('student.'):
                    new_dict[key[8:]] = checkpoint_model[key]
                else:
                    pass
            checkpoint_model = new_dict

        state_dict = model.state_dict()
        for k in ['head.weight', 'head.bias']:
            if k in checkpoint_model and checkpoint_model[k].shape != state_dict[k].shape:
                print(f"Removing key {k} from pretrained checkpoint")
                del checkpoint_model[k]

        all_keys = list(checkpoint_model.keys())
        for key in all_keys:
            if "relative_position_index" in key:
                checkpoint_model.pop(key)

        utils.load_state_dict(model, checkpoint_model, prefix=args.model_prefix)


以上から、私がもし分類数を変えたければ、modeling_finetune.py内の分類ヘッドを変えればいいことが判明した。

12/4 次に、今日は異なるデータセットに対して処理を行うために、データセットを処理する関数について調べよう。